<a href="https://colab.research.google.com/github/Tannongma/SCM.275x/blob/main/SCM_275x_Pricing_Decisions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SCM.275x - Advanced Supply Chain Systems Planning and Network Design
# **Pricing Decisions**

### *Before starting, make sure to save a copy of this notebook to your Google Drive!*

## **Initialization**

In [ ]:
# Install necessary packages if they are not already installed

!pip install gurobipy   # Gurobi optimization solver
!pip install pandas     # Pandas for data analysis and manipulation
!pip install folium     # Folium for creating interactive maps
!pip install geopy      # Geopy for computing distances and working with geographic data


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 41.9 MB/s eta 0:00:00


In [ ]:
# Import all required packages

import pandas as pd                   # For data manipulation and analysis
import gurobipy as grb                # Gurobi optimization library for solving mathematical models
import folium                         # For creating interactive maps
import folium.plugins as plugins      # Additional plugins for folium
from folium import Choropleth         # Import Choropleth from folium for creating choropleth maps
from geopy.distance import geodesic   # For calculating geodesic distances between two points
import geopandas as gpd               # Import GeoPandas for working with geospatial data and extending pandas functionality to handle geometry

## **Helper functions**

### **Plotting nodes on a map**

In [ ]:
# Defining a function to plot nodes on a map using folium

def plot_nodes(map,                         # Folium map object to plot the nodes on
               nodes,                       # Dictionary of node objects where each node contains attributes like latitude and longitude
               icon,                        # Icon symbol to use for the markers on the map
               active_color,                # Color of the marker icon for active nodes
               background_color,            # Background color of the marker icon
               inactive_color = 'grey',     # Color of the marker icon for inactive nodes
               ):

    # Loop through each node in the dictionary
    for node in nodes.values():

        # Create a folium marker
        marker = folium.Marker(
            location=[node.lat, node.lon],              # Set the marker's location
            popup = (node.ID + "-" + node.name),        # Create a marker popup with the node ID and name
            icon=plugins.BeautifyIcon(                  # Create a marker's icon
                icon=icon,
                icon_shape="circle",
                text_color=active_color if node.active == True else inactive_color,
                border_color=active_color if node.active == True else inactive_color,
                background_color=background_color,
            )
        )

        # Add a folium marker to the map
        marker.add_to(map)


### **Computing geodesic distance**

In [ ]:
# Defining a function for computing geodesic distances between two locations

def compute_geodesic_distance(origin,       # Origin node object
                              destination,  # Destination node
                              unit='km'):   # Unit ('km' or 'mi')

    # Extract coordinates (latitude and longitude) from origin and destination
    origin_coordinates = [origin.lat, origin.lon]
    destination_coordinates = [destination.lat, destination.lon]

    # Compute distance based on the specified unit
    if unit == 'km':
        distance = geodesic(origin_coordinates, destination_coordinates).km  # Compute distance in kilometers
    elif unit == 'mi':
        distance = geodesic(origin_coordinates, destination_coordinates).mi  # Compute distance in miles

    return distance  # Return the calculated distance


### **Plotting geographies - choropleth map**

In [ ]:
# Defining a function for creating a choropleth map

def plot_geographies_choropleth(map,                  # Folium map object to plot the geographies on
                                geographies,          # Dictionary of objects containing geographical data
                                attribute_to_plot,    # Object attribute used to create the map
                                fill_color="YlGn"):   # Color used


    # Create a pandas DataFrame from the geographies for easier plotting
    # Note: getattr() function allows to retrieve an attribute by name; for example: getattr(demand_zone_1, 'selling_price') provides the selling price of the demand_zone_1

    data = pd.DataFrame({
        'ID': [geo.ID for geo in geographies.values()],
        attribute_to_plot: [getattr(geo, attribute_to_plot) for geo in geographies.values()],
        'geometry': [geo.geometry for geo in geographies.values()],
    })


    # Convert the DataFrame to GeoJSON format
    # GeoJSON format is used to represent geographic data in a JSON-like structure. It includes the type of geometry (e.g., Polygon, Point) and the coordinates.

    geo_json_data = {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "geometry": geo.geometry.__geo_interface__,  # Convert the geometry to GeoJSON format using the __geo_interface__ attribute of the geometry object
                "properties": {"ID": geo.ID, attribute_to_plot: getattr(geo, attribute_to_plot)}
            }
            for geo in geographies.values()
        ]
    }

    # Add a Choropleth layer to the map
    # Choropleth is a type of map where areas are shaded based on the value of a variable, in this case, the 'attribute_to_plot'.

    Choropleth(
        geo_data=geo_json_data,  # Use the GeoJSON data for geometry
        name="choropleth",
        data=data,
        columns=["ID", attribute_to_plot],
        key_on="feature.properties.ID",
        fill_color=fill_color,
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=attribute_to_plot,
    ).add_to(map)


### **Plotting geographies - categorical map**

In [ ]:
def plot_geographies_by_category(map,                   # Folium map object to plot the geographies on
                                 geographies,           # Dictionary of objects containing geographical data
                                 attribute_to_plot,     # Object attribute used to create the map
                                 category_colors):      # Color used

    # Create a pandas DataFrame from the geographies for easier plotting
    data = pd.DataFrame({
        'ID': [geo.ID for geo in geographies.values()],
        attribute_to_plot: [getattr(geo, attribute_to_plot) for geo in geographies.values()],
        'geometry': [geo.geometry for geo in geographies.values()],
    })

    # Convert the DataFrame to GeoJSON format
    geo_json_data = {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "geometry": geo.geometry.__geo_interface__,  # Convert geometry to GeoJSON format
                "properties": {"ID": geo.ID, attribute_to_plot: getattr(geo, attribute_to_plot)}
            }
            for geo in geographies.values()
        ]
    }

    # Add polygons to the map and color them based on the specified category color mapping
    for _, row in data.iterrows():
        color = category_colors.get(row[attribute_to_plot], "gray")  # Default to 'gray' if category not in category_colors
        folium.GeoJson(
            row['geometry'].__geo_interface__,
            style_function=lambda feature, color=color: {
                'fillColor': color,
                'color': 'black',
                'weight': 0.5,
                'fillOpacity': 0.7,
                'lineOpacity': 0.2,
            },
            tooltip=f"ID: {row['ID']}, {attribute_to_plot}: {row[attribute_to_plot]}"
        ).add_to(map)

    # Add a legend for the categories
    legend_html = """
    <div style="position: fixed;
                bottom: 50px; left: 50px; width: 150px; height: 150px;
                background-color: white; z-index:9999; font-size:14px;">
    <h4>Legend</h4>
    <ul style="list-style-type: none; padding: 0;">"""
    for cat, color in category_colors.items():
        legend_html += f"<li><span style='background-color:{color}; padding:5px;'></span> {cat}</li>"
    legend_html += "</ul></div>"

    map.get_root().html.add_child(folium.Element(legend_html))


## **Data setup and preprocessing**

### **Nodes**

#### Reading input files

In [ ]:
# File containing containing demand zone data
demand_zones_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/pricing_decisions/demand_zones.json'

# Loading demand zone data into a pandas DataFrame
demand_zones_df = gpd.read_file(demand_zones_file)

# Displaying the first few rows of the DataFrame to verify the data
demand_zones_df.head()

,ID,county,state,lon,lat,max_demand,demand_pct_rank,geometry
0,d_06021,Glenn County,California,-122.391281,39.597389,28.339,0.345361,"POLYGON ((-122.91179 39.79948, -122.04647 39.7..."
1,d_49053,Washington County,Utah,-113.504682,37.280037,197.680,0.706186,"POLYGON ((-114.05247 37.60478, -113.5878 37.60..."
2,d_53063,Spokane County,Washington,-117.404704,47.621276,549.690,0.865979,"POLYGON ((-117.82095 47.82578, -117.69777 47.8..."
3,d_04013,Maricopa County,Arizona,-112.492806,33.347626,1500.000,0.971649,"POLYGON ((-113.33505 33.37747, -113.33331 34.0..."
4,d_53045,Mason County,Washington,-123.186178,47.349623,68.166,0.525773,"POLYGON ((-123.50592 47.60462, -122.95062 47.6..."


In [ ]:
# File containing demand and pricing data
demand_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/pricing_decisions/demand_data.csv'

# Loading demand data into a pandas DataFrame
demand_data_df = pd.read_csv(demand_data_file)

# Displaying the first few rows of the DataFrame to verify the data
demand_data_df.head()

,demand_zone_ID,pricing_level,selling_price,demand
0,d_04001,low,100,65.4320
1,d_04001,medium,110,52.3456
2,d_04001,high,120,39.2592
3,d_04003,low,100,125.6630
4,d_04003,medium,110,100.5304


In [ ]:
# Visualizing max demand for a specific demand zone
demand_zones_df[demand_zones_df['ID'] == 'd_04001']

,ID,county,state,lon,lat,max_demand,demand_pct_rank,geometry
98,d_04001,Apache County,Arizona,-109.489218,35.394691,65.432,0.515464,"POLYGON ((-110.00068 36.99797, -109.49534 36.9..."


In [ ]:
# File containing warehouse data
warehouse_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/pricing_decisions/warehouses.csv'

# Loading warehouse data into a pandas DataFrame
warehouses_df = pd.read_csv(warehouse_data_file)

# Displaying the first few rows of the DataFrame to verify the data
warehouses_df.head()

,ID,name,lat,lon,fixed_cost,capacity,landed_cost
0,w1,San Bernandino,34.095457,-117.232554,400000,20000,49
1,w2,Portland,45.617900,-122.599070,280000,20000,48
2,w3,Oakland,37.823133,-122.268155,400000,20000,50
3,w4,San Diego,32.790078,-117.069334,240000,20000,52


#### Definition of Classes

In [ ]:
# Class representing a DemandZone object

class DemandZone():
    def __init__(self, ID, max_demand, lat, lon, state, geometry):
        self.ID = ID                  # DemandZone's ID
        self.max_demand = max_demand  # DemandZone's maximum demand value
        self.lat = lat                # DemandZone's latitude
        self.lon = lon                # DemandZone's longitude
        self.geometry = geometry      # DemandZone's geometry (geospatial data)

        self.demand = dict()          # Initializing an empty dictionnary to store information on demand
        self.selling_price = dict()   # Initializing an empty dictionnary to store information on selling price


In [ ]:
# Class representing a Warehouse object

class Warehouse():
    def __init__(self, ID, name, lat, lon, fixed_cost, capacity, landed_cost):
        self.ID = ID                        # Warehouse's ID
        self.name = name                    # Warehouse's name
        self.lat = lat                      # Warehouse's latitude
        self.lon = lon                      # Warehouse's longitude
        self.fixed_cost = fixed_cost        # Warehouse's fixed cost
        self.capacity = capacity            # Warehouse's capacity
        self.landed_cost = landed_cost      # Warehouse's landed cost

        self.active = True                  # Initializing node as active


#### Creating node objects

In [ ]:
# Initializing an empty dictionary to store node objects
nodes = dict()

In [ ]:
# Creating a dictionary of demand zone objects

demand_zones = dict()
for i, row in demand_zones_df.iterrows():
    demand_zones[row['ID']] = DemandZone(ID = row['ID'],                    # DemandZone's ID
                                         max_demand = row['max_demand'],    # DemandZone's demand value
                                         lat = row['lat'],                  # DemandZone's latitude
                                         lon = row['lon'],                  # DemandZone's longitude
                                         state = row['state'],              # DemandZone's longitude
                                         geometry = row['geometry'])        # DemandZone's geometry (geospatial data)

# Merging the demand zones dictionary into the existing nodes dictionary
nodes = {**nodes, **demand_zones}

In [ ]:
# Associating demand data and pricing data to demand zones
for i, row in demand_data_df.iterrows():
    demand_zones[row['demand_zone_ID']].demand[row['pricing_level']] = row['demand']
    demand_zones[row['demand_zone_ID']].selling_price[row['pricing_level']] = row['selling_price']


In [ ]:
# Creating a dictionary of warehouse objects
warehouses = dict()
for i, row in warehouses_df.iterrows():
    warehouses[row['ID']] = Warehouse(ID = row['ID'],                     # Warehouse's ID
                                    name = row['name'],                   # Warehouse's name
                                    lat = row['lat'],                     # Warehouse's latitude
                                    lon = row['lon'],                     # Warehouse's longitude
                                    fixed_cost = row['fixed_cost'],       # Warehouse's fixed cost
                                    capacity = row['capacity'],           # Warehouse's fixed cost
                                    landed_cost = row['landed_cost'])     # Warehouse's landed cost

# Merging the warehouses dictionary into the existing nodes dictionary
nodes = {**nodes, **warehouses}

#### Visualizing node objects

In [ ]:
# Create a new map
map = folium.Map([40, -100.0], zoom_start=5)

# Plot warehouse locations with a warehouse icon, blue color, and white background
plot_nodes(map=map, nodes=warehouses, icon='warehouse', active_color='blue', background_color='white')

# Plot demand zones on a map
plot_geographies_choropleth(map=map, geographies=demand_zones, attribute_to_plot='max_demand', fill_color = 'YlGnBu')

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map


### **Arcs**

#### Arc distances

In [ ]:
# Creating a dictionary containing distances between warehouses and demand_zones
distances = dict()
for w, warehouse in warehouses.items():
  for d, demand_zone in demand_zones.items():
      distances[w, d] = compute_geodesic_distance(origin = warehouse, destination = demand_zone, unit = 'km')

#### Arc costs

In [ ]:
cost_unit_km = 0.1    # Cost per unit per kilometer from warehouse to demand zones

# Creating a dictionary containing unit costs between between warehouses and demand zones
unit_cost = dict()

for w, warehouse in warehouses.items():                                                        # Iterate over warehouses
    for d, demand_zone in demand_zones.items():                                                # Iterate over demand zones
        unit_cost[w, d] = distances[w, d] * cost_unit_km                                       # Calculate unit cost as distance multiplied by cost per km (warehouse to demand zone)


### **Pricing levels**

In [ ]:
# Defining a list of pricing levels to consider in the model
pricing_levels = ['low', 'medium', 'high']

## **Optimization model**

### **Creating and solving the optimization model**

In [ ]:
# Initializing the model
model = grb.Model("Pricing decisions")

# Creating decision variables

# Create binary location variables for each warehouse
wh_location_vars = dict()
for w, warehouse in warehouses.items():
    wh_location_vars[w] = model.addVar(vtype=grb.GRB.BINARY,
                                       name="wh_location{0}".format(w))


# Create continuous flow variables for each warehouse and demand zone pair
flow_vars = dict()
for w, warehouse in warehouses.items():  # Iterate over items in warehouses
    for d, demand_zone in demand_zones.items():  # Iterate over items in demand_zones
        flow_vars[w, d] = model.addVar(vtype=grb.GRB.CONTINUOUS,
                                       name="flow{0}_{1}".format(w, d))

# Create binary allocation variables for each demand zone and pricing level pair
allocation_price_level_vars = dict()
for d, demand_zone in demand_zones.items():
    for p in pricing_levels:
        allocation_price_level_vars[d, p] = model.addVar(vtype=grb.GRB.BINARY,
                                                         name="allocation_price_level_{0}_{1}".format(d, p))


# Creating the objective function

# Revenue calculation based on allocation, demand, and selling price
revenue = grb.quicksum(demand_zone.selling_price[p] * allocation_price_level_vars[d, p] * demand_zone.demand[p]
                       for d, demand_zone in demand_zones.items() for p in pricing_levels)

# Fixed cost of operating warehouses
fixed_cost_warehouses = grb.quicksum(warehouse.fixed_cost * wh_location_vars[w] for w, warehouse in warehouses.items())

# Transportation cost
transportation_cost = grb.quicksum(unit_cost[w, d] * flow_vars[w, d] for w, warehouse in warehouses.items() for d, demand_zone in demand_zones.items())

# Landed cost
landed_cost = grb.quicksum(flow_vars[w, d] * warehouse.landed_cost for w, warehouse in warehouses.items() for d, demand_zone in demand_zones.items())

# Profit
profit = revenue - fixed_cost_warehouses - transportation_cost - landed_cost

# Set the objective to maximize profit
model.setObjective(profit, grb.GRB.MAXIMIZE)

# Adding demand constraints
for d, demand_zone in demand_zones.items():
    model.addConstr(grb.quicksum(flow_vars[w, d] for w, warehouse in warehouses.items()) ==
                    grb.quicksum(allocation_price_level_vars[d, p] * demand_zone.demand[p] for p in pricing_levels))

# Adding capacity constraints
for w, warehouse in warehouses.items():
    model.addConstr(grb.quicksum(flow_vars[w, d] for d, demand_zone in demand_zones.items()) <= wh_location_vars[w] * warehouse.capacity)

# Adding pricing level constraint
for d, demand_zone in demand_zones.items():
    model.addConstr(grb.quicksum(allocation_price_level_vars[d, p] for p in pricing_levels) == 1)

# Optimize the model
model.optimize()

# Set the activation decisions of the warehouses
for w, warehouse in warehouses.items():
  warehouse.active = True if wh_location_vars[w].X == 1 else False

# Set the pricing level of the demand zone to the selected pricing level
for d, demand_zone in demand_zones.items():
  demand_zone.pricing_level = "none"
  for p in pricing_levels:
    if allocation_price_level_vars[d, p].X == 1:
      demand_zone.pricing_level = p


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 392 rows, 1362 columns and 2720 nonzeros
Model fingerprint: 0x5b533545
Variable types: 776 continuous, 586 integer (586 binary)
Coefficient statistics:
  Matrix range     [4e-01, 2e+04]
  Objective range  [5e+01, 4e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -1789319.526
Presolve removed 0 rows and 194 columns
Presolve time: 0.01s
Presolved: 392 rows, 1168 columns, 2332 nonzeros
Variable types: 776 continuous, 392 integer (392 binary)
Found heuristic solution: objective -1692158.766

Root relaxation: objective 8.489940e+05, 416 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth 

In [ ]:
# Create a new map
map = folium.Map([40, -100.0], zoom_start=5)

# Plot warehouse locations with a warehouse icon, blue color, and white background
plot_nodes(map=map, nodes=warehouses, icon='warehouse', active_color='blue', background_color='white')

category_colors = {'none': 'grey',
                    'low': 'green',
                    'medium': 'yellow',
                    'high': 'orange'}


# Plot demand zones on a map and show the chosen pricing level
plot_geographies_by_category(map, geographies=demand_zones, attribute_to_plot="pricing_level", category_colors = category_colors)

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map
